In [ ]:
import torch
from datasets import load_dataset, load_metric
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, TrainingArguments, Trainer
import wandb
import numpy as np
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [ ]:
# Load the XSum dataset
dataset = load_dataset("xsum")

wandb.login()

In [ ]:
# Set the model name and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [ ]:
def preprocess_batch(batch):
    input_texts = ["summarize: " + doc for doc in batch["document"]]
    target_texts = batch["summary"]

    source = tokenizer(input_texts, max_length=512, truncation=True, padding='max_length', return_tensors="pt")
    target = tokenizer(target_texts, max_length=150, truncation=True, padding='max_length', return_tensors="pt")

    return {
        "input_ids": source["input_ids"].tolist(),
        "attention_mask": source["attention_mask"].tolist(),
        "labels": target["input_ids"].tolist(),
    }

tokenized_dataset = dataset.map(preprocess_batch, remove_columns=["document", "summary"], batched=True)

In [ ]:
# Load the T5 model
config = T5Config.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, config=config)


In [ ]:
# Set up training and evaluation arguments
training_args = TrainingArguments(
    output_dir="./t5_xsum",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb", 
)


In [ ]:
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset["validation"],
)

In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
# Save the fine-tuned model
trainer.save_model("./t5_xsum_finetuned")

In [ ]:
def generate_summary_transformer(article):
    model = T5ForConditionalGeneration.from_pretrained('t5-small')
    tokenizer = T5Tokenizer.from_pretrained('t5-small')

    inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, num_return_sequences=1, max_length=150, no_repeat_ngram_size=2, min_length=30, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def evaluate_model(generate_summary_function):
    rouge = Rouge()
    bleu_score = 0
    dataset = load_dataset('xsum', split='test')
    predictions = []
    references = []

    for example in dataset:
        article = example["document"]
        summary = generate_summary_function(article)
        predictions.append(summary)
        references.append(example["summary"])

    rouge_scores = rouge.compute(predictions=predictions, references=references, rouge_types=["rouge1", "rouge2", "rougeL"])

    # Compute BLEU score
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.split()
        ref_tokens = ref.split()
        bleu_score += sentence_bleu([ref_tokens], pred_tokens, smoothing_function=SmoothingFunction().method1)

    bleu_score = bleu_score / len(predictions)

    return rouge_scores, bleu_score

# Evaluate the Transformer-based model
transformer_rouge_scores, transformer_bleu_score = evaluate_model(generate_summary_transformer)
print("Transformer Rouge Scores:", transformer_rouge_scores)
print("Transformer BLEU Score:", transformer_bleu_score)